In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

# my bibs
from cnn_net_bib import * 
from cnn_plot_bib import *
from cnn_data_import import *

IMPORTED NETWORK FUNCTIONS
IMPORTED PLOTTING FUNCTIONS


In [2]:
folder = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/'
path_melanoma = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/melanoma/'
path_rotated_melanoma = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/rotated_melanoma/'
path_non_melanoma = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/non_melanoma/'
img_names = os.listdir(folder)
img_path2 = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/all/'
img_path = '/Users/indp/Dropbox/rotations/polavieja/data/good_datasets/isic-archive_ss/all2/'

In [3]:
data = read_data_sets(train_dir = folder, num_classes = 2)

Chosen size of the image: (448, 598)
 Progress:  0.10659104244083709Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok, skipping.
Could not read file - it's ok,

KeyboardInterrupt: 

In [ ]:
img_shape = (448, 598)
img_size_flat = img_shape[0] * img_shape[1]
num_channels = 1 # grey scale
num_classes = 2

In [ ]:
print('train')
plot_images(data.train.images, 3, img_shape)
plt.show()
print('test')
plot_images(data.test.images, 3, img_shape)
plt.show()
print('validation')
plot_images(data.validation.images, 3, img_shape)
plt.show()

In [ ]:
# Loading the MNIST data
#from tensorflow.examples.tutorials.mnist import input_data
#data = input_data.read_data_sets('data/MNIST/', one_hot=True)

#data.test.cls = np.argmax(data.test.labels, axis=1)

# MNIST data dimensionality

In [ ]:
# Convolutional Layer 1.
filter_size1 = 6          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 6          
num_filters2 = 32        

filter_size3 = 6          
num_filters3 = 64         

filter_size4 = 6          
num_filters4 = 128       

# Fully-connected layer.
fc_size = 128             

# cnn

In [ ]:
# placeholder variables 

x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_shape[0], img_shape[1], num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)
weight = tf.placeholder(tf.float32, shape=(), name='weight')

In [ ]:
# Network architecture - Tensorflow definitions

layer_conv1, weights_conv1 = new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [ ]:
layer_conv3, weights_conv3 = new_conv_layer(input=layer_conv2,
                   num_input_channels=num_filters2,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=True)

In [ ]:
layer_conv4, weights_conv4 = new_conv_layer(input=layer_conv3,
                   num_input_channels=num_filters3,
                   filter_size=filter_size4,
                   num_filters=num_filters4,
                   use_pooling=True)

layer_flat, num_features = flatten_layer(layer_conv4)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [ ]:
# network predictions

y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, dimension=1)

# cost function and optimization measures

# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
# using weighted cross entropy due to dataset inbalance

cross_entropy = tf.nn.weighted_cross_entropy_with_logits(targets = y_true, 
                                                         logits = layer_fc2, 
                                                         pos_weight = weight)

cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# performance measures

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#recall = tf.metrics.recall(labels = y_true, predictions = y_pred)
#precision = tf.metrics.precision(labels = y_true, predictions = y_pred)
#recall = 

In [ ]:
# running the network

session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(tf.local_variables_initializer())
train_batch_size = 50

# optimization algorithm
total_iterations = 0
def optimize(num_iterations):

    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch, weight_batch = data.train.next_batch(train_batch_size)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch,
                           weight: weight_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict = feed_dict_train)
        
        # non_optimal way of calculating recall and precision
        yt = y_true_batch
        yt_cls = np.asarray(tf.argmax(yt, dimension = 1).eval(feed_dict={x: x_batch}, session=session))
        
        yp = y_pred.eval(feed_dict={x: x_batch}, session=session)
        yp_cls = np.asarray(tf.argmax(yp, dimension = 1).eval(feed_dict={x: x_batch}, session=session))
        
        tp = np.sum(np.asarray([1 if e == d and e == 1 else 0 for e,d in zip(yt_cls, yp_cls)]))
        tn = np.sum(np.asarray([1 if e == d and e == 0 else 0 for e,d in zip(yt_cls, yp_cls)]))
        fp = np.sum(np.asarray([1 if e != d and e == 0 else 0 for e,d in zip(yt_cls, yp_cls)]))
        fn = np.sum(np.asarray([1 if e != d and e == 1 else 0 for e,d in zip(yt_cls, yp_cls)]))
        
        print('TP:', tp, 'TN:', tn, 'FP:', fp, 'FN:', fn)
        print('Recall: ', tp/float(tp+fn), 'Precision: ', tp/float(tp+fp))

        # Print status every n iterations.
        if i % 1 == 0:
            # Calculate the accuracy on the training-set.
            acc  = session.run(accuracy ,feed_dict=feed_dict_train)

            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))
            # print('Recall: ', rec, 'Precision: ', prec) # not usefull
    
    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
optimize(num_iterations = 20)

# testing performance

In [ ]:
#### Split the test-set into smaller batches of this size.
test_batch_size = 20

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(data.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = data.test.images[i:j, :]

        # Get the associated labels.
        labels = data.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = data.test.cls

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

In [ ]:
data.test.cls = np.argmax(data.test.labels, axis=1)

In [ ]:
print_test_accuracy()

In [ ]:
data.train.next_batch(train_batch_size)[2]

In [ ]:
optimize(num_iterations = 1)